In [45]:
from matplotlib import pyplot as plt


def ishow(img,
          cmap='viridis',
          title='',
          fig_size=(8,6),
          colorbar=False,
          interpolation='none'):
    ' Function `ishow` displays an image in a new window. '
    
    if img.min() < 0:
        img -= img.min()
        img /= img.max()
    
    extent = (0, img.shape[1], img.shape[0], 0)
    fig, ax = plt.subplots(figsize=fig_size)
    pcm = ax.imshow(img,
              extent=extent,
              cmap=cmap,
              interpolation=interpolation)
    
    ax.set_frame_on(False)
    plt.title(title)
    plt.tight_layout()
    if colorbar:
        
        fig.colorbar(pcm, orientation='vertical')
    plt.show()
    

In [46]:
from pathlib import Path


PATH = Path('data', 'data_seg_public')
town_dir = PATH / 'img'
mask_dir = PATH / 'mask'
# [ len(list(img_dir.iterdir())) for img_dir in sorted(list(town_dir.iterdir()))]

In [47]:
# [ len(list(img_dir.iterdir())) for img_dir in sorted(list(mask_dir.iterdir()))]

In [48]:
import tqdm
import pandas as pd
import glob

# samples = []
# n_samples = len(list(img_dir.glob('*')))
# towns = sorted(list(town_dir.iterdir()))
# img_files = [list(img.iterdir()) for img in towns]
# img_files = sorted([item for sublist in img_files for item in sublist])
# ann_files = list(ann_dir.iterdir())
# for img_path, ann_path in zip(img_files, ann_files):
#     # Create a dictionary for the current sample
#     sample = {
#         'img_path': img_path,
#         'ann_path': ann_path
#     }
#     # Append the dictionary to the samples list
#     samples.append(sample)



# df = pd.DataFrame(samples)
# df.loc[0]['ann_path']
# img_files

In [63]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transforms_train = A.Compose([
                        #  augmentation 
                        #  A.ColorJitter(brightness=.5, contrast=.5, saturation=.5, hue=.1),
                        #  A.GaussNoise(),
                        #  A.HorizontalFlip(p=0.5),
                        #  A.ShiftScaleRotate(rotate_limit=30, p=1, border_mode=0, value=0, mask_value=3),
                        #  A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
                            
                        #  # preprocessing
                        #  A.SmallestMaxSize (1024),
                        #  A.CenterCrop(512, 1024),
                        #  A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        #  ToTensorV2(),
                        ]   
                    )

In [64]:
from sklearn.model_selection import train_test_split
from dataset import SampleDataset
from network import SampleModel


In [66]:

img_files = glob.glob("{}/*/*.png".format(town_dir))
mask_files = glob.glob("{}/*/*.png".format(mask_dir))
df = pd.DataFrame({'img': img_files, 'mask': mask_files})
train_df, valid_df = train_test_split(df, test_size=.3, random_state=2)

train_df, valid_df = train_test_split(df, test_size=.3, random_state=2)
traindataset = SampleDataset(dataset_df=train_df, transforms=transforms_train)


model = SampleModel(num_class=8)
IDX = 2285
x, y = traindataset[IDX]
# pred = model(x)
print("wtf")
ishow(x)



AttributeError: 'numpy.ndarray' object has no attribute 'type'